# Data Celaning / Processing and Feature Engineering

During Exploratory Data Analysis there were no missing values or duplicate items so that i will leave out.

For now I'm also not gonna do outlier detection since outliers dont usually ruin the performance of the recommender system and I havent seen extreme cases but later if the model is not giving good results i might look into getting rid of users with too little and/or too much ratings.

In [2]:
pip install numpy pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [4]:
#Ignore warnings so it doesnt clutter
import warnings
warnings.filterwarnings("ignore")

## Ratings

In [5]:
ratings_column_names = ['UserID', 'MovieID', 'Rating', 'Timestamp']

ratings_path = '../data/ml-1m-(raw)/ratings.dat'
ratings = pd.read_csv(ratings_path, header=None, sep='::', engine='python', names=ratings_column_names, encoding="latin-1")

print(ratings.head())

   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


In [6]:
avg_ratings = ratings.groupby('MovieID')['Rating'].agg(['mean', 'count'])
ratings = ratings.merge(avg_ratings, on='MovieID', how='left')
ratings.rename(columns={'mean': 'Avg_Rating', 'count': 'Rating_Count'}, inplace=True)

# Scale average ratings
ratings['Avg_Rating'] = MinMaxScaler().fit_transform(ratings[['Avg_Rating']])
# Scale rating count
ratings['Rating_Count'] = MinMaxScaler().fit_transform(ratings[['Rating_Count']])

print(ratings.head())

   UserID  MovieID  Rating  Timestamp  Avg_Rating  Rating_Count
0       1     1193       5  978300760    0.847681      0.503064
1       1      661       3  978302109    0.616190      0.152903
2       1      914       3  978301968    0.788522      0.185293
3       1     3408       4  978300275    0.715970      0.383426
4       1     2355       5  978824291    0.713594      0.496644


In [7]:
# Dropping timestamps because we arent affected by time
ratings = ratings.drop(columns=['Timestamp'])

print(ratings.head())

   UserID  MovieID  Rating  Avg_Rating  Rating_Count
0       1     1193       5    0.847681      0.503064
1       1      661       3    0.616190      0.152903
2       1      914       3    0.788522      0.185293
3       1     3408       4    0.715970      0.383426
4       1     2355       5    0.713594      0.496644


## Users

In [8]:
users_column_names = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]

users_path = '../data/ml-1m-(raw)/users.dat'
users = pd.read_csv(users_path, header=None, sep='::', engine="python", names=users_column_names, encoding="latin-1")

print(users.head())

   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455


In [9]:
# Extract first 5 digits ONLY from ZIP+4 formats (exactly 10 characters with hyphen)
users['Zip-code'] = users['Zip-code'].where(
    ~users['Zip-code'].str.match(r'^\d{5}-\d{4}$', na=False),
    users['Zip-code'].str[:5]
)

In [10]:
#Convert zips to states
zips_path = '../data/zip_to_state/zip_to_state.csv'
zip_to_state = pd.read_csv('../data/zip_to_state/zip_to_state.csv', dtype={'zip': str})
#needs to be read in with dtype str since it assumes zip codes are numbers otherwise

print(zip_to_state.head())

     zip    state
0  47236  Indiana
1  47870  Indiana
2  47851  Indiana
3  47337  Indiana
4  47435  Indiana


In [11]:
# Check for leading zeros or formatting differences
print(users['Zip-code'].str.len().value_counts())
print(zip_to_state['zip'].str.len().value_counts())

Zip-code
5    6025
6      11
7       3
9       1
Name: count, dtype: int64
zip
5    36063
Name: count, dtype: int64


In [12]:
merged_users = users.merge(zip_to_state, left_on='Zip-code', right_on='zip', how='left')

print(merged_users.head())

   UserID Gender  Age  Occupation Zip-code    zip          state
0       1      F    1          10    48067  48067       Michigan
1       2      M   56          16    70072  70072      Louisiana
2       3      M   25          15    55117  55117      Minnesota
3       4      M   45           7    02460  02460  Massachusetts
4       5      M   25          20    55455  55455      Minnesota


In [13]:
merged_users = merged_users.rename(columns={'state': 'State'})

#drop zips, not needed anymore

merged_users = merged_users[['UserID', 'Gender', 'Age', 'Occupation', 'State']]

print(merged_users.head())

   UserID Gender  Age  Occupation          State
0       1      F    1          10       Michigan
1       2      M   56          16      Louisiana
2       3      M   25          15      Minnesota
3       4      M   45           7  Massachusetts
4       5      M   25          20      Minnesota


In [14]:
merged_users.info()
# I have 204 missing states but I will keep them as is some and use a model that handles it
# Deleting these would be a waste since we have very little data the location is probably not that important

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6279 entries, 0 to 6278
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   UserID      6279 non-null   int64 
 1   Gender      6279 non-null   object
 2   Age         6279 non-null   int64 
 3   Occupation  6279 non-null   int64 
 4   State       6075 non-null   object
dtypes: int64(3), object(2)
memory usage: 245.4+ KB


In [15]:
# One-Hot Encoding to handle states.

# These are the Categorical colmns
cat_cols = ['Gender', 'Occupation', 'State']
cat_df = merged_users[cat_cols].astype(str)  # To string jst to be safe

# Initialize and fit one-hot encoder
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop first avoids dummy trap
encoded_array = encoder.fit_transform(cat_df)

# Get feature names
feature_names = encoder.get_feature_names_out(input_features=cat_cols)

# Create new dataframe with encoded features
encoded_users = pd.concat([
    merged_users[['UserID', 'Age']],
    pd.DataFrame(encoded_array, columns=feature_names)
], axis=1)

print(encoded_users)

      UserID  Age  Gender_M  Occupation_1  Occupation_10  Occupation_11  \
0          1    1       0.0           0.0            1.0            0.0   
1          2   56       1.0           0.0            0.0            0.0   
2          3   25       1.0           0.0            0.0            0.0   
3          4   45       1.0           0.0            0.0            0.0   
4          5   25       1.0           0.0            0.0            0.0   
...      ...  ...       ...           ...            ...            ...   
6274    6036   25       0.0           0.0            0.0            0.0   
6275    6037   45       0.0           1.0            0.0            0.0   
6276    6038   56       0.0           1.0            0.0            0.0   
6277    6039   45       0.0           0.0            0.0            0.0   
6278    6040   25       1.0           0.0            0.0            0.0   

      Occupation_12  Occupation_13  Occupation_14  Occupation_15  ...  \
0               0.0       

## Movies

In [16]:
movies_column_names = ["MovieID", "Title", "Genres"]

movies_path = '../data/ml-1m-(raw)/movies.dat'
movies = pd.read_csv(movies_path, header=None, sep='::', engine="python", names=movies_column_names, encoding="latin-1")

# Extract release year from titles (e.g., "Toy Story (1995)" → 1995)
movies['Year'] = movies['Title'].str.extract(r'\((\d{4})\)').astype(float)

# Create decade feature (works better than year)
movies['Decade'] = (movies['Year'] // 10 * 10).astype(int)

movies = movies.drop(columns=['Year'])

print(movies.head())

   MovieID                               Title                        Genres  \
0        1                    Toy Story (1995)   Animation|Children's|Comedy   
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2        3             Grumpier Old Men (1995)                Comedy|Romance   
3        4            Waiting to Exhale (1995)                  Comedy|Drama   
4        5  Father of the Bride Part II (1995)                        Comedy   

   Decade  
0    1990  
1    1990  
2    1990  
3    1990  
4    1990  


In [17]:
# One-Hot Ecoding to handle genres. 
# Some movies have one, some have more so this works well this way.

# Get all unique genres
all_genres = set()
movies['Genres'].str.split('|').apply(all_genres.update)

encoded_movies = movies.copy()

# Create binary columns
for genre in all_genres:
    encoded_movies[f'Genre_{genre}'] = movies['Genres'].str.contains(genre).astype(int)

# Min-max scale Decade:
encoded_movies['Decade'] = MinMaxScaler().fit_transform(encoded_movies[['Decade']])

#These columns dont matter for us
encoded_movies = encoded_movies.drop(columns=['Genres', 'Title']) 

print(encoded_movies.columns)

print(encoded_movies.head(20))

Index(['MovieID', 'Decade', 'Genre_Adventure', 'Genre_Comedy', 'Genre_Action',
       'Genre_Romance', 'Genre_Thriller', 'Genre_Horror', 'Genre_Sci-Fi',
       'Genre_War', 'Genre_Fantasy', 'Genre_Crime', 'Genre_Western',
       'Genre_Drama', 'Genre_Animation', 'Genre_Musical', 'Genre_Film-Noir',
       'Genre_Documentary', 'Genre_Mystery', 'Genre_Children's'],
      dtype='object')
    MovieID    Decade  Genre_Adventure  Genre_Comedy  Genre_Action  \
0         1  0.888889                0             1             0   
1         2  0.888889                1             0             0   
2         3  0.888889                0             1             0   
3         4  0.888889                0             1             0   
4         5  0.888889                0             1             0   
5         6  0.888889                0             0             1   
6         7  0.888889                0             1             0   
7         8  0.888889                1             0 

## Content-Based DataFrame Merging

In [18]:
content_df = ratings.merge(encoded_movies, on='MovieID', how='left')

print(content_df.columns)
print(content_df.head(10))

Index(['UserID', 'MovieID', 'Rating', 'Avg_Rating', 'Rating_Count', 'Decade',
       'Genre_Adventure', 'Genre_Comedy', 'Genre_Action', 'Genre_Romance',
       'Genre_Thriller', 'Genre_Horror', 'Genre_Sci-Fi', 'Genre_War',
       'Genre_Fantasy', 'Genre_Crime', 'Genre_Western', 'Genre_Drama',
       'Genre_Animation', 'Genre_Musical', 'Genre_Film-Noir',
       'Genre_Documentary', 'Genre_Mystery', 'Genre_Children's'],
      dtype='object')
   UserID  MovieID  Rating  Avg_Rating  Rating_Count    Decade  \
0       1     1193       5    0.847681      0.503064  0.666667   
1       1      661       3    0.616190      0.152903  0.888889   
2       1      914       3    0.788522      0.185293  0.555556   
3       1     3408       4    0.715970      0.383426  1.000000   
4       1     2355       5    0.713594      0.496644  0.888889   
5       1     1197       3    0.825928      0.676102  0.777778   
6       1     1287       5    0.777699      0.205136  0.444444   
7       1     2804       5  

In [19]:
content_df.to_csv("../data/prepared-data/content-based-features.csv", index=False)

## Hypbrid User-Based and Content-Based DataFrame Merging

In [20]:
# First merge ratings with user features
user_ratings = ratings.merge(encoded_users, on='UserID', how='left')

# Then merge with movie features
collab_df = user_ratings.merge(encoded_movies, on='MovieID', how='left')

print(collab_df.columns)
#print(collab_df.head(10))

Index(['UserID', 'MovieID', 'Rating', 'Avg_Rating', 'Rating_Count', 'Age',
       'Gender_M', 'Occupation_1', 'Occupation_10', 'Occupation_11',
       'Occupation_12', 'Occupation_13', 'Occupation_14', 'Occupation_15',
       'Occupation_16', 'Occupation_17', 'Occupation_18', 'Occupation_19',
       'Occupation_2', 'Occupation_20', 'Occupation_3', 'Occupation_4',
       'Occupation_5', 'Occupation_6', 'Occupation_7', 'Occupation_8',
       'Occupation_9', 'State_Alaska', 'State_Arizona', 'State_Arkansas',
       'State_California', 'State_Colorado', 'State_Connecticut',
       'State_Delaware', 'State_District of Columbia', 'State_Florida',
       'State_Georgia', 'State_Hawaii', 'State_Idaho', 'State_Illinois',
       'State_Indiana', 'State_Iowa', 'State_Kansas', 'State_Kentucky',
       'State_Louisiana', 'State_Maine', 'State_Maryland',
       'State_Massachusetts', 'State_Michigan', 'State_Minnesota',
       'State_Mississippi', 'State_Missouri', 'State_Montana',
       'State_Neb

In [21]:
#Scaling the Age since that is the only numerical column (ratings also numerical but probably fine since its 1-5)
scaler = MinMaxScaler()
collab_df['Age'] = scaler.fit_transform(collab_df[['Age']])

print(collab_df.head(10))

   UserID  MovieID  Rating  Avg_Rating  Rating_Count  Age  Gender_M  \
0       1     1193       5    0.847681      0.503064  0.0       0.0   
1       1      661       3    0.616190      0.152903  0.0       0.0   
2       1      914       3    0.788522      0.185293  0.0       0.0   
3       1     3408       4    0.715970      0.383426  0.0       0.0   
4       1     2355       5    0.713594      0.496644  0.0       0.0   
5       1     1197       3    0.825928      0.676102  0.0       0.0   
6       1     1287       5    0.777699      0.205136  0.0       0.0   
7       1     2804       5    0.809726      0.394222  0.0       0.0   
8       1      594       4    0.711337      0.222352  0.0       0.0   
9       1      919       4    0.811991      0.501021  0.0       0.0   

   Occupation_1  Occupation_10  Occupation_11  ...  Genre_Fantasy  \
0           0.0            1.0            0.0  ...              0   
1           0.0            1.0            0.0  ...              0   
2          

In [22]:
collab_df.to_csv("../data/prepared-data/user-content-based-features.csv", index=False)